In [1]:
import xarray_beam as xb
ds, _ = xb.open_zarr('gcs://era5_preprocess/1440x720/2006-12-31_2011-12-31.zarr')

In [6]:
print(ds['geopotential'][23][2].values)

[[131331.98 131333.22 131328.61 131321.23 131306.78 131286.48 131271.11
  131260.66 131255.12 131261.27 131274.19 131280.03 131274.48 131256.66
  131227.14 131201.   131200.69 131227.75 131268.64 131308.62 131334.75
  131342.14 131339.67 131339.06 131343.67 131354.75 131371.05 131387.03]
 [131473.12 131473.73 131470.05 131462.67 131449.14 131430.08 131409.17
  131390.72 131385.48 131396.56 131407.62 131411.62 131417.16 131414.1
  131386.72 131351.67 131339.06 131350.75 131375.03 131412.25 131453.75
  131477.12 131479.28 131477.12 131480.5  131489.11 131503.27 131519.56]
 [131615.48 131615.48 131612.11 131605.66 131596.12 131580.75 131556.77
  131533.7  131526.62 131534.62 131542.61 131548.77 131561.69 131567.52
  131545.08 131509.72 131490.66 131488.5  131490.03 131508.48 131554.61
  131598.27 131612.72 131612.72 131618.25 131628.72 131642.55 131659.16]
 [131758.47 131758.16 131755.7  131751.1  131745.25 131732.03 131709.27
  131688.06 131677.61 131678.53 131684.98 131694.52 131705.58 

In [10]:
print(original[0][1][-1])
print(original[1][1][-1])

tensor([0.4935, 0.1810, 0.0266,  ..., 0.9930, 0.0333, 0.8678])
tensor([-0.0131, -0.6380, -0.9467,  ...,  0.9856, -0.9335,  0.7351])


In [3]:
%load_ext autoreload
%autoreload 2
from models.VariableAnalyzer.datasets.dataset import CustomDataset
dataset = CustomDataset(original[0], original[1], 4 * 7)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
print(dataset[0])

(tensor([[1.8671e-01, 1.7526e-01, 1.6456e-01,  ..., 3.7757e-01, 1.8123e-01,
         2.7494e-02],
        [1.6623e-01, 1.6128e-01, 1.5663e-01,  ..., 1.4949e-01, 4.8082e-02,
         0.0000e+00],
        [1.3278e-01, 1.2662e-01, 1.2060e-01,  ..., 9.5946e-02, 2.5475e-02,
         0.0000e+00],
        ...,
        [9.9636e-01, 9.9600e-01, 9.9784e-01,  ..., 9.5954e-01, 2.2724e-01,
         7.4940e-01],
        [9.9821e-01, 9.9800e-01, 9.7642e-01,  ..., 9.5332e-01, 4.9272e-02,
         8.2218e-01],
        [9.7311e-01, 9.8831e-01, 7.5980e-01,  ..., 9.4597e-01, 8.4211e-04,
         9.1442e-01]]), tensor([[2.2350e-01, 2.1749e-01, 2.1065e-01,  ..., 3.6762e-01, 1.7511e-01,
         1.9818e-02],
        [1.1157e-01, 1.0535e-01, 9.9633e-02,  ..., 1.5458e-01, 4.8664e-02,
         0.0000e+00],
        [1.1850e-01, 1.1255e-01, 1.0675e-01,  ..., 1.0239e-01, 2.7377e-02,
         0.0000e+00],
        ...,
        [7.8148e-01, 7.4497e-01, 7.3062e-01,  ..., 9.2654e-01, 9.6733e-01,
         5.1592e-02],
 

In [4]:
from torch.utils.data import DataLoader, Dataset

dataloader = DataLoader(dataset, 32, shuffle=True)

In [5]:
print(len(dataloader))

228


In [6]:
for i, batch in enumerate(dataloader):
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227


In [50]:
%load_ext autoreload
%autoreload 2
from models.VariableAnalyzer.models.model import VariableAnalyzer

model = VariableAnalyzer(100, 4 * 7, 1450, 10, 3, 3).to(device)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [57]:
for i, batch in enumerate(dataloader):
    print(model.get_time_seq(batch).)
    if i == 0:
        break

torch.Size([32, 2800])


In [26]:
def get_time_seq(src: torch.Tensor):
        time_seq = []
        for i in range(4 * 7):
            seq = [i for _ in range(100)]
            time_seq.extend(seq)
        return torch.tensor([time_seq for _ in range(src.size(0))])


for i, batch in enumerate(dataloader):
    print()
    if i == 0:
        break

(slice(0, 3, None), 1)


TypeError: list indices must be integers or slices, not tuple